In [5]:
from IPython.parallel import Client
rc=Client()#获取客户端控制
dview=rc[:]#<DirectView [0, 1]>
dview.block=True#设置dview.block
print(rc.ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


/home/srthu2/anaconda3/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@img11')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [6]:

from magenta.music import note_sequence_io
from magenta.music import sequences_lib
from magenta.music import melodies_lib
from magenta.music import Melody
from magenta.music import musicxml_parser
from magenta.music import musicxml_reader

from magenta.music import lead_sheets_lib, chords_lib
import os
import copy
import sys
lib_path = os.path.abspath(os.path.join('.'))
sys.path.append(lib_path)

In [7]:
dview.execute('from magenta.music import note_sequence_io')
dview.execute('from magenta.music import musicxml_reader')
dview.execute('from magenta.music import musicxml_parser')
dview.execute('from magenta.music import sequences_lib')
dview.execute('from magenta.music import melodies_lib')
dview.execute('from magenta.music import Melody')
dview.execute('from magenta.music import lead_sheets_lib, chords_lib')

dview.execute('import os')
dview.execute('import copy')
dview.execute('import sys')
#dview.execute('lib_path = os.path.abspath(os.path.join(\'.\'))')
# dview.execute('sys.path.append(\'/home/srthu2/dongyuanliang/\')')
# dview.execute('from my_musicxml_reader import my_musicxml_file_to_sequence_proto')

<AsyncResult: execute:finished>

## api for processing files and get ns

In [201]:
def get_files(folder):
    file_list = []
    for filename in os.listdir(folder):
        if filename.startswith('.'):
            continue
        if filename.endswith('.mxl') or filename.endswith('.xml'):
            file_list.append(os.path.join(folder, filename))
        elif os.path.isdir(os.path.join(folder, filename)):
            file_list.extend(get_files(os.path.join(folder, filename)))
    return file_list

def mxl2xml(mxl_files):
    for filename in xml_files:
        mxlzip = zipfile.ZipFile(filename)

dview.execute('''
    
# coding: utf-8

# In[1]:

from magenta.protobuf import music_pb2

# Shortcut to CHORD_SYMBOL annotation type.
CHORD_SYMBOL = music_pb2.NoteSequence.TextAnnotation.CHORD_SYMBOL


class MusicXMLConversionError(Exception):
  """MusicXML conversion error handler."""
  pass


def my_musicxml_to_sequence_proto(musicxml_document):
  """Convert MusicXML file contents to a tensorflow.magenta.NoteSequence proto.

  Converts a MusicXML file encoded as a string into a
  tensorflow.magenta.NoteSequence proto.

  Args:
    musicxml_document: A parsed MusicXML file.
        This file has been parsed by class MusicXMLDocument

  Returns:
    A tensorflow.magenta.NoteSequence proto.

  Raises:
    MusicXMLConversionError: An error occurred when parsing the MusicXML file.
  """
  sequence = music_pb2.NoteSequence()

  # Standard MusicXML fields.
  sequence.source_info.source_type = (
      music_pb2.NoteSequence.SourceInfo.SCORE_BASED)
  sequence.source_info.encoding_type = (
      music_pb2.NoteSequence.SourceInfo.MUSIC_XML)
  sequence.source_info.parser = (
      music_pb2.NoteSequence.SourceInfo.MAGENTA_MUSIC_XML)

  # Populate header.
  sequence.ticks_per_quarter = musicxml_document.midi_resolution

  # Populate time signatures.
  musicxml_time_signatures = musicxml_document.get_time_signatures()
  for musicxml_time_signature in musicxml_time_signatures:
    time_signature = sequence.time_signatures.add()
    time_signature.time = musicxml_time_signature.time_position
    time_signature.numerator = musicxml_time_signature.numerator
    time_signature.denominator = musicxml_time_signature.denominator

  # Populate key signatures.
  musicxml_key_signatures = musicxml_document.get_key_signatures()
  for musicxml_key in musicxml_key_signatures:
    key_signature = sequence.key_signatures.add()
    key_signature.time = musicxml_key.time_position
    # The Key enum in music.proto does NOT follow MIDI / MusicXML specs
    # Convert from MIDI / MusicXML key to music.proto key
    music_proto_keys = [11, 6, 1, 8, 3, 10, 5, 0, 7, 2, 9, 4, 11, 6, 1]
    key_signature.key = music_proto_keys[musicxml_key.key + 7]
    if musicxml_key.mode == "major":
      key_signature.mode = key_signature.MAJOR
    elif musicxml_key.mode == "minor":
      key_signature.mode = key_signature.MINOR

  # Populate tempo changes.
  musicxml_tempos = musicxml_document.get_tempos()
  for musicxml_tempo in musicxml_tempos:
    tempo = sequence.tempos.add()
    tempo.time = musicxml_tempo.time_position
    tempo.qpm = musicxml_tempo.qpm

  # Populate notes from each MusicXML part across all voices
  # Unlike MIDI import, notes are not sorted
  sequence.total_time = musicxml_document.total_time_secs
  for part_index, musicxml_part in enumerate(musicxml_document.parts):
    part_info = sequence.part_infos.add()
    part_info.part = part_index
    part_info.name = musicxml_part.score_part.part_name

    for musicxml_measure in musicxml_part.measures:
      for musicxml_note in musicxml_measure.notes:
        if musicxml_note.is_in_chord:
          continue
        if not musicxml_note.is_rest:
          note = sequence.notes.add()
          note.part = part_index
          note.voice = musicxml_note.voice
          note.instrument = musicxml_note.midi_channel
          note.program = musicxml_note.midi_program
          note.start_time = musicxml_note.note_duration.time_position

          # Fix negative time errors from incorrect MusicXML
          if note.start_time < 0:
            note.start_time = 0

          note.end_time = note.start_time + musicxml_note.note_duration.seconds
          note.pitch = musicxml_note.pitch[1]  # Index 1 = MIDI pitch number
          note.velocity = musicxml_note.velocity

          durationratio = musicxml_note.note_duration.duration_ratio()
          note.numerator = durationratio.numerator
          note.denominator = durationratio.denominator
        else:
          note = sequence.notes.add()
          note.part = part_index
          note.voice = musicxml_note.voice
          note.instrument = musicxml_note.midi_channel
          note.program = musicxml_note.midi_program
          note.start_time = musicxml_note.note_duration.time_position

          # Fix negative time errors from incorrect MusicXML
          if note.start_time < 0:
            note.start_time = 0

          note.end_time = note.start_time + musicxml_note.note_duration.seconds
          note.pitch = 0  # Index 1 = MIDI pitch number
          note.velocity = musicxml_note.velocity

          durationratio = musicxml_note.note_duration.duration_ratio()
          note.numerator = durationratio.numerator
          note.denominator = durationratio.denominator

  musicxml_chord_symbols = musicxml_document.get_chord_symbols()
  for musicxml_chord_symbol in musicxml_chord_symbols:
    text_annotation = sequence.text_annotations.add()
    text_annotation.time = musicxml_chord_symbol.time_position
    text_annotation.text = musicxml_chord_symbol.get_figure_string()
    text_annotation.annotation_type = CHORD_SYMBOL

  return sequence


def my_musicxml_file_to_sequence_proto(musicxml_file):
  """Converts a MusicXML file to a tensorflow.magenta.NoteSequence proto.

  Args:
    musicxml_file: A string path to a MusicXML file.

  Returns:
    A tensorflow.magenta.Sequence proto.

  Raises:
    MusicXMLConversionError: Invalid musicxml_file.
  """
  from magenta.protobuf import music_pb2
  from magenta.music import note_sequence_io
  from magenta.music import sequences_lib
  from magenta.music import melodies_lib
  from magenta.music import Melody
  from magenta.music import musicxml_parser
  from magenta.music import musicxml_reader
  try:
    musicxml_document = musicxml_parser.MusicXMLDocument(musicxml_file)
  except musicxml_parser.MusicXMLParseException as e:
    raise MusicXMLConversionError(e)
  return my_musicxml_to_sequence_proto(musicxml_document)




'''
)

def my_mxl2ns(full_file_path):

    sequence = my_musicxml_file_to_sequence_proto(full_file_path)
    sequence.filename = os.path.abspath(full_file_path)
    sequence.id = note_sequence_io.generate_note_sequence_id(
      sequence.filename, sequence.collection_name, 'musicxml')
 
    return sequence


# working

In [9]:
mxl_files = get_files('/home/srthu2/dongyuanliang/dataset/0524')
print('Collect {} mxl files'.format(len(mxl_files)))

Collect 2404 mxl files


In [174]:
machine_work_files = get_files('/home/srthu2/dongyuanliang/machine_work/bleu')
print('Collect {} mxl files'.format(len(machine_work_files)))

Collect 8001 mxl files


In [202]:
machine_sequence_list = dview.map(my_mxl2ns, machine_work_files)

print('Collect {} ns files'.format(len(machine_sequence_list)))


Collect 8001 ns files


In [204]:
cmp = 1e-3

for my_ns in machine_sequence_list:
    
    for my_note in my_ns.notes:
        a = my_note.numerator * 1.0 / my_note.denominator
        if(my_note.numerator == 0):
            print(my_ns.filename)
            print(my_note)
            break
        b = (my_note.end_time - my_note.start_time) / 2 * my_ns.tempos[0].qpm / 120.0
        if(abs(a - b) > cmp ):
            print(my_note)
            flag = 0
            for i in range(1, 8):
                for j in range(1, 33):
                    aa = float(i) / j
                    if(abs(aa - b) < cmp):
                        my_note.numerator = i
                        my_note.denominator = j
                        flag = 1
                        break
                if(flag == 1):
                    break


In [11]:
cmp = 1e-3
for my_ns in note_sequence_list:
    for my_note in my_ns.notes:
        a = my_note.numerator * 1.0 / my_note.denominator
        if(my_note.numerator == 0):
            print(my_ns.filename)
            print(my_note)
        b = (my_note.end_time - my_note.start_time) / 2
        if(abs(a - b) > cmp):
            flag = 0
            for i in range(1, 8):
                for j in range(1, 33):
                    aa = float(i) / j
                    if(abs(aa - b) < cmp):
                        my_note.numerator = i
                        my_note.denominator = j
                        flag = 1
                        break
                if(flag == 1):
                    break
            print(my_note)


velocity: 64
start_time: 35.0
end_time: 36.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 41.0
end_time: 42.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 47.0
end_time: 48.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 59.0
end_time: 60.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 65.0
end_time: 66.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 71.0
end_time: 72.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 24.0
end_time: 25.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 25.5
end_time: 27.0
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 15.0
end_time: 16.0
numerator: 1
denominator: 2
voice: 1

velocity: 64
start_time: 102.5
end_time: 104.0
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 48.0
end_time: 49.5
numerator: 3
denominator: 4
voice: 1

velocity: 64
start_time: 26.0
end_time: 27.0
numerator: 1
denominator: 2
voice: 1

ve

In [222]:

for my_ns in machine_sequence_list[100:200]:
    if(len(my_ns.time_signatures) > 1):
        print(my_ns.filename)
        print(my_ns.time_signatures)
    if not (my_ns.time_signatures[0].numerator == 4 and my_ns.time_signatures[0].denominator == 4):
        print(my_ns.filename+"heng")   
    if(len(my_ns.key_signatures) > 1):
        print(my_ns.filename+"he")   
    if(len(my_ns.part_infos) > 1):
        flag = 0
        for note in my_ns.notes:
            if(note.pitch >= 60 and note.part == 1):
                flag = 1
        if(flag == 1):
            print(my_ns.filename)
        

/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/2-1527854006.736409.xml
[numerator: 4
denominator: 4
, time: 36.0
numerator: 1
denominator: 2
]
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/2-1527856427.0689523.xml
[numerator: 4
denominator: 4
, time: 36.0
numerator: 1
denominator: 2
]
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/2-1527854699.7891235.xml
[numerator: 4
denominator: 4
, time: 36.0
numerator: 1
denominator: 2
]
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/7-1527856603.3220134.xml
[numerator: 4
denominator: 4
, time: 50.0
numerator: 1
denominator: 2
]
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/2-1527856826.1019244.xml
[numerator: 4
denominator: 4
, time: 36.0
numerator: 1
denominator: 2
]
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/2-1527856033.467847.xml
[numerator: 4
denominator: 4
, time: 36.0
numerator: 1
denominator

In [12]:
ns_ts1 = list(filter(lambda x: len(x.time_signatures) == 1, note_sequence_list))
ns_ts44 = list(filter(lambda x: x.time_signatures[0].numerator == 4 and x.time_signatures[0].denominator == 4, ns_ts1))
ns_ks1_ts44 = list(filter(lambda x: len(x.key_signatures) == 1 and not '有点甜' in x.filename, ns_ts44))
print(len(ns_ks1_ts44))
print(ns_ks1_ts44[-1])

2135
id: "/id/musicxml//7a821db4daebb97bc049378eb6b42aea6826975f"
filename: "1851 - \347\224\260\346\235\221\351\272\273\345\245\210\345\256\237 - \343\201\202\343\202\212\343\201\265\343\202\214\343\201\237\346\234\252\346\235\245\343\201\270.xml"
ticks_per_quarter: 220
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
  key: A_SHARP
}
tempos {
  qpm: 120.0
}
notes {
  pitch: 62
  velocity: 64
  end_time: 0.25
  numerator: 1
  denominator: 8
  voice: 1
}
notes {
  pitch: 60
  velocity: 64
  start_time: 0.25
  end_time: 0.375
  numerator: 1
  denominator: 16
  voice: 1
}
notes {
  pitch: 62
  velocity: 64
  start_time: 0.375
  end_time: 0.5
  numerator: 1
  denominator: 16
  voice: 1
}
notes {
  pitch: 62
  velocity: 64
  start_time: 0.5
  end_time: 0.75
  numerator: 1
  denominator: 8
  voice: 1
}
notes {
  pitch: 60
  velocity: 64
  start_time: 0.75
  end_time: 1.0
  numerator: 1
  denominator: 8
  voice: 1
}
notes {
  pitch: 62
  velocity: 64
  start_time: 1.0
  e

In [ ]:
ns_ts1 = list(filter(lambda x: len(x.time_signatures) == 1, note_sequence_list))
ns_ts44 = list(filter(lambda x: x.time_signatures[0].numerator == 4 and x.time_signatures[0].denominator == 4, ns_ts1))
ns_ks1_ts44 = list(filter(lambda x: len(x.key_signatures) == 1 and not '有点甜' in x.filename, ns_ts44))
print(len(ns_ks1_ts44))
print(ns_ks1_ts44[-1])

In [13]:
for my_ns in ns_ks1_ts44:
    transpose = my_ns.key_signatures[0].key
    for my_note in my_ns.notes:
        if(my_note.pitch > 0):
            my_note.pitch -= transpose
print(ns_ks1_ts44[-1])

id: "/id/musicxml//7a821db4daebb97bc049378eb6b42aea6826975f"
filename: "1851 - \347\224\260\346\235\221\351\272\273\345\245\210\345\256\237 - \343\201\202\343\202\212\343\201\265\343\202\214\343\201\237\346\234\252\346\235\245\343\201\270.xml"
ticks_per_quarter: 220
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
  key: A_SHARP
}
tempos {
  qpm: 120.0
}
notes {
  pitch: 52
  velocity: 64
  end_time: 0.25
  numerator: 1
  denominator: 8
  voice: 1
}
notes {
  pitch: 50
  velocity: 64
  start_time: 0.25
  end_time: 0.375
  numerator: 1
  denominator: 16
  voice: 1
}
notes {
  pitch: 52
  velocity: 64
  start_time: 0.375
  end_time: 0.5
  numerator: 1
  denominator: 16
  voice: 1
}
notes {
  pitch: 52
  velocity: 64
  start_time: 0.5
  end_time: 0.75
  numerator: 1
  denominator: 8
  voice: 1
}
notes {
  pitch: 50
  velocity: 64
  start_time: 0.75
  end_time: 1.0
  numerator: 1
  denominator: 8
  voice: 1
}
notes {
  pitch: 52
  velocity: 64
  start_time: 1.0
  end_ti

In [14]:
from fractions import Fraction
new2_ns_ks1_ts44 = []

for my_ns in ns_ks1_ts44:
    new_notes = []
    for note in my_ns.notes:
        new_notes.append((note.pitch, Fraction(note.numerator, note.denominator)))

    new2_ns_ks1_ts44.append(new_notes)
print(new2_ns_ks1_ts44[-1])

[(52, Fraction(1, 8)), (50, Fraction(1, 16)), (52, Fraction(1, 16)), (52, Fraction(1, 8)), (50, Fraction(1, 8)), (52, Fraction(1, 8)), (50, Fraction(1, 16)), (52, Fraction(1, 16)), (52, Fraction(1, 8)), (50, Fraction(1, 16)), (52, Fraction(1, 16)), (52, Fraction(1, 8)), (50, Fraction(1, 16)), (52, Fraction(1, 16)), (52, Fraction(1, 8)), (50, Fraction(1, 8)), (48, Fraction(1, 8)), (50, Fraction(1, 8)), (52, Fraction(1, 8)), (48, Fraction(1, 8)), (55, Fraction(3, 8)), (57, Fraction(1, 8)), (55, Fraction(1, 8)), (48, Fraction(1, 4)), (48, Fraction(1, 8)), (53, Fraction(1, 8)), (52, Fraction(1, 8)), (53, Fraction(1, 8)), (55, Fraction(1, 8)), (52, Fraction(1, 4)), (50, Fraction(1, 4)), (52, Fraction(3, 8)), (52, Fraction(1, 8)), (59, Fraction(1, 8)), (60, Fraction(1, 16)), (59, Fraction(1, 16)), (59, Fraction(1, 8)), (60, Fraction(1, 8)), (52, Fraction(1, 8)), (50, Fraction(1, 8)), (50, Fraction(1, 8)), (50, Fraction(1, 8)), (50, Fraction(1, 8)), (52, Fraction(1, 4)), (48, Fraction(1, 8)),

In [223]:
machine_ns = []

for my_ns in machine_sequence_list:
    new_notes = []
    for note in my_ns.notes:
        if(note.part == 0):
            new_notes.append((note.pitch, Fraction(note.numerator, note.denominator)))

    machine_ns.append(new_notes)
print(len(machine_ns))
print(machine_ns[-1])

8001
[(74, Fraction(1, 8)), (72, Fraction(1, 8)), (74, Fraction(1, 8)), (72, Fraction(1, 8)), (71, Fraction(1, 8)), (69, Fraction(1, 8)), (71, Fraction(1, 8)), (69, Fraction(1, 8)), (71, Fraction(1, 8)), (72, Fraction(1, 8)), (72, Fraction(1, 8)), (74, Fraction(1, 8)), (74, Fraction(1, 8)), (72, Fraction(1, 8)), (69, Fraction(1, 4)), (65, Fraction(1, 8)), (72, Fraction(1, 8)), (72, Fraction(1, 8)), (74, Fraction(1, 8)), (74, Fraction(1, 8)), (72, Fraction(1, 8)), (69, Fraction(1, 8)), (69, Fraction(1, 8)), (67, Fraction(1, 8)), (72, Fraction(1, 8)), (72, Fraction(1, 8)), (67, Fraction(1, 8)), (67, Fraction(1, 8)), (69, Fraction(1, 8)), (67, Fraction(1, 4)), (67, Fraction(3, 4)), (0, Fraction(1, 4)), (67, Fraction(3, 8)), (67, Fraction(1, 8)), (67, Fraction(1, 4)), (67, Fraction(1, 4)), (67, Fraction(1, 4)), (76, Fraction(1, 8)), (67, Fraction(1, 8)), (67, Fraction(1, 4)), (67, Fraction(1, 4)), (72, Fraction(1, 8)), (71, Fraction(1, 8)), (72, Fraction(1, 8)), (70, Fraction(1, 8)), (71, 

In [224]:
print(machine_sequence_list[-1])

id: "/id/musicxml//d3e2ae41fd7a7c2f9caf811a2282a8815d6853f2"
filename: "/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_lookback/48/8-1530261739.144305.xml"
ticks_per_quarter: 220
time_signatures {
  numerator: 4
  denominator: 4
}
time_signatures {
  time: 59.52000000000007
  numerator: 1
  denominator: 2
}
key_signatures {
}
tempos {
  qpm: 125.0
}
notes {
  pitch: 74
  velocity: 64
  end_time: 0.24
  numerator: 1
  denominator: 8
  instrument: 1
  program: 1
  voice: 1
}
notes {
  pitch: 72
  velocity: 64
  start_time: 0.24
  end_time: 0.48
  numerator: 1
  denominator: 8
  instrument: 1
  program: 1
  voice: 1
}
notes {
  pitch: 74
  velocity: 64
  start_time: 0.48
  end_time: 0.72
  numerator: 1
  denominator: 8
  instrument: 1
  program: 1
  voice: 1
}
notes {
  pitch: 72
  velocity: 64
  start_time: 0.72
  end_time: 0.96
  numerator: 1
  denominator: 8
  instrument: 1
  program: 1
  voice: 1
}
notes {
  pitch: 71
  velocity: 64
  start_time: 0.96
  end_time: 1.2
  numerat

In [44]:
dview.execute('from fractions import Fraction')
def get_final(my_ns):
    end_times = [Fraction(0, 1)] * len(my_ns)
    end_times[0] = my_ns[0][1]
    
    for i in range(1, len(my_ns)):
        end_times[i]= end_times[i-1] + my_ns[i][1]
    
    cur = 0
    index = 0
    totlen = len(my_ns)
    final_seq = []
    unit = Fraction(1, 32)
    while(cur < totlen):
        if(index * unit >= end_times[cur] - my_ns[cur][1] and (index+1) * unit <= end_times[cur]):
            final_seq.append(my_ns[cur][0])
            index += 1
        elif(end_times[cur] - my_ns[cur][1] < index * unit and index * unit < end_times[cur] and end_times[cur] < (index+1) * unit):
            final_seq.append(my_ns[cur][0])
            index += 1
            cur += 1
        else:
            cur += 1
    return final_seq

In [226]:
final_machine_ns = dview.map(get_final, machine_ns)
print(final_machine_ns[-1])

[74, 74, 74, 74, 72, 72, 72, 72, 74, 74, 74, 74, 72, 72, 72, 72, 71, 71, 71, 71, 69, 69, 69, 69, 71, 71, 71, 71, 69, 69, 69, 69, 71, 71, 71, 71, 72, 72, 72, 72, 72, 72, 72, 72, 74, 74, 74, 74, 74, 74, 74, 74, 72, 72, 72, 72, 69, 69, 69, 69, 69, 69, 69, 69, 65, 65, 65, 65, 72, 72, 72, 72, 72, 72, 72, 72, 74, 74, 74, 74, 74, 74, 74, 74, 72, 72, 72, 72, 69, 69, 69, 69, 69, 69, 69, 69, 67, 67, 67, 67, 72, 72, 72, 72, 72, 72, 72, 72, 67, 67, 67, 67, 67, 67, 67, 67, 69, 69, 69, 69, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 0, 0, 0, 0, 0, 0, 0, 0, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 76, 76, 76, 76, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 72, 72, 72, 72, 71, 71, 71, 71, 72, 72, 72, 72, 70, 70, 70, 70, 71, 71, 71, 71, 64, 64, 64, 64, 64, 64, 64, 64,

In [115]:
final_ns_ks1_ts44 = dview.map(get_final, new2_ns_ks1_ts44)

print(final_ns_ks1_ts44[-1])

[52, 52, 52, 52, 50, 50, 52, 52, 52, 52, 52, 52, 50, 50, 50, 50, 52, 52, 52, 52, 50, 50, 52, 52, 52, 52, 52, 52, 50, 50, 52, 52, 52, 52, 52, 52, 50, 50, 52, 52, 52, 52, 52, 52, 50, 50, 50, 50, 48, 48, 48, 48, 50, 50, 50, 50, 52, 52, 52, 52, 48, 48, 48, 48, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 57, 57, 57, 57, 55, 55, 55, 55, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 53, 53, 53, 53, 52, 52, 52, 52, 53, 53, 53, 53, 55, 55, 55, 55, 52, 52, 52, 52, 52, 52, 52, 52, 50, 50, 50, 50, 50, 50, 50, 50, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 59, 59, 59, 59, 60, 60, 59, 59, 59, 59, 59, 59, 60, 60, 60, 60, 52, 52, 52, 52, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 52, 52, 52, 52, 52, 52, 52, 48, 48, 48, 48, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 53, 53, 53, 53, 52, 52, 52, 52, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,

In [227]:
print(len(final_machine_ns[-1]))
for my_ns in final_machine_ns:
    i = -1
    while(my_ns[i] == 0):
        i -= 1
    last_pitch = my_ns[i]
    print(last_pitch)
    i += 1
    while(i < 0):
        my_ns[i] = last_pitch
        i += 1
    c = len(my_ns) % 32
    if(c > 0):
        my_ns += [last_pitch] * (32-c)
    my_ns.append(0)


1008
65
60
69
72
77
74
65
69
60
60
67
72
72
67
64
64
64
64
64
64
69
69
60
72
60
64
60
64
55
64
65
62
55
67
74
62
74
64
64
69
69
67
76
72
74
72
64
67
64
57
55
67
67
60
71
60
67
67
52
60
64
72
72
72
74
60
74
67
69
60
50
64
57
72
79
76
55
69
64
64
64
60
60
64
67
64
69
62
72
72
62
72
74
69
60
76
62
76
64
62
60
64
67
67
74
60
62
67
72
60
72
62
72
72
64
81
67
67
76
67
67
74
55
64
62
74
74
69
60
60
63
60
71
60
65
67
67
69
72
60
65
67
60
64
60
64
67
72
65
67
59
60
65
64
74
71
67
64
76
64
69
67
69
60
67
74
60
72
62
62
65
62
72
69
60
71
67
72
72
72
71
72
72
62
62
55
60
64
74
74
62
67
74
60
67
69
69
60
76
67
60
62
76
62
69
64
69
67
71
60
60
72
60
64
60
64
69
62
70
69
76
62
64
60
55
72
72
60
62
67
60
62
71
65
67
64
67
72
67
72
67
60
64
60
64
60
55
77
64
57
62
64
76
62
67
76
67
62
72
76
64
76
72
72
62
64
76
60
72
67
64
64
67
76
67
69
60
57
77
72
71
64
72
72
64
60
60
60
69
57
67
76
67
64
69
60
60
74
55
59
81
64
64
64
60
62
60
67
74
62
74
64
62
65
76
67
72
62
72
57
67
69
76
71
64
64
67
67
76
67
67
74

53
72
71
64
72
65
72
60
76
68
67
72
55
67
53
57
72
67
62
57
55
64
72
74
76
60
62
65
60
62
77
62
74
55
59
67
62
72
60
79
57
64
67
67
65
62
69
71
71
74
69
67
65
74
76
55
62
64
64
60
60
67
59
67
71
60
62
79
53
64
53
60
64
77
70
67
60
65
69
71
74
55
62
64
75
60
77
67
67
68
67
62
67
68
62
72
65
65
72
79
69
64
71
69
74
67
62
65
60
53
72
71
55
67
67
72
65
64
50
76
69
72
77
64
62
65
67
60
62
62
65
55
60
72
60
60
67
72
60
67
67
74
62
64
72
60
60
62
65
62
64
72
67
74
76
67
60
67
76
67
53
64
74
69
62
66
69
68
72
79
67
60
60
57
65
67
59
67
69
60
60
72
69
67
69
74
64
79
72
64
67
59
69
69
74
65
65
55
65
72
60
69
60
62
74
71
67
72
72
52
60
65
64
60
62
55
62
55
69
62
71
76
69
60
65
69
72
72
69
55
67
74
72
60
77
64
69
67
60
64
64
74
65
57
62
62
67
62
76
60
71
64
69
67
64
64
79
76
71
60
59
64
71
72
76
67
60
72
64
79
64
72
62
67
55
72
64
72
69
60
67
67
59
60
64
72
65
77
67
67
67
62
62
69
60
76
74
67
69
64
72
60
69
64
64
64
72
69
74
69
73
65
64
69
64
60
64
74
71
77
60
64
67
79
67
74
72
62
69
71
72
65
64
6

53
59
52
67
71
55
62
67
69
74
64
62
62
55
55
74
62
62
62
64
71
62
62
67
60
64
71
79
62
62
62
62
60
64
55
71
77
62
60
62
74
59
74
74
62
62
69
64
72
57
60
72
62
67
67
71
77
74
55
62
67
71
67
71
67
67
64
71
55
62
60
62
62
62
60
64
52
71
64
67
65
64
60
55
60
76
59
72
74
62
64
65
67
53
67
62
67
55
62
64
62
62
64
64
62
62
55
59
67
72
65
62
72
55
60
62
59
60
62
67
67
79
74
62
72
64
55
67
55
60
67
74
64
64
62
60
60
60
72
74
57
69
65
64
60
62
60
67
60
60
60
71
60
55
76
72
67
60
71
72
60
61
72
67
65
72
76
60
72
57
60
62
71
59
72
71
60
72
60
67
67
72
67
67
62
64
69
62
55
67
67
62
64
72
64
65
81
74
60
72
72
64
68
67
60
62
64
72
57
65
72
60
71
67
71
69
64
62
67
69
69
62
64
69
57
74
71
64
74
60
76
60
62
74
60
60
76
71
74
64
60
62
69
64
69
69
55
71
69
71
64
72
71
69
69
62
62
62
62
62
67
64
74
69
74
65
60
60
69
65
62
71
65
65
69
76
60
60
60
60
69
64
67
72
72
65
59
64
62
76
64
60
76
79
72
64
60
65
74
67
65
74
74
60
74
72
67
65
57
64
62
72
69
69
62
63
62
67
69
74
67
62
62
64
67
52
64
60
60
71
67
67
74
6

In [277]:
dview.execute(
'''
import collections
import math


def get_ngrams(segment, minn, maxn):
    """Extracts all n-grams upto a given maximum order from an input segment.
    Args:
    segment: text segment from which n-grams will be extracted.
    max_order: maximum length in tokens of the n-grams returned by this
        methods.
    Returns:
    The Counter containing all n-grams upto max_order in segment
    with a count of how many times each n-gram occurred.
    """
    types = []
    ngram_counts = set()
    for order in range(minn, maxn + 1):
        cur = 0
        prelen = len(ngram_counts)
        for i in range(0, len(segment) - order + 1):
            if(cur < i):
                cur += 1
            while(cur < i + order and segment[cur][0] <= 0):
                cur += 1
            
            modify = 0
            if(cur < i + order):
                while(segment[cur][0] + modify < 60):
                    modify += 12
                while(segment[cur][0] + modify >= 72):
                    modify -= 12
            ngram = tuple((x[0] if x[0] <= 0 else x[0] + modify, x[1]) for x in segment[i:i+order])
            ngram_counts.add(ngram)
        types.append(len(ngram_counts) - prelen)
    return ngram_counts, types
''')

<AsyncResult: execute:finished>

In [276]:
import collections
import math


def get_ngrams(segment, minn, maxn):
    """Extracts all n-grams upto a given maximum order from an input segment.
    Args:
    segment: text segment from which n-grams will be extracted.
    max_order: maximum length in tokens of the n-grams returned by this
        methods.
    Returns:
    The Counter containing all n-grams upto max_order in segment
    with a count of how many times each n-gram occurred.
    """
    types = []
    ngram_counts = set()
    for order in range(minn, maxn + 1):
        cur = 0
        prelen = len(ngram_counts)
        for i in range(0, len(segment) - order + 1):
            if(cur < i):
                cur += 1
            while(cur < i + order and segment[cur][0] <= 0):
                cur += 1
            
            modify = 0
            if(cur < i + order):
                while(segment[cur][0] + modify < 60):
                    modify += 12
                while(segment[cur][0] + modify >= 72):
                    modify -= 12
            ngram = tuple((x[0] if x[0] <= 0 else x[0] + modify, x[1]) for x in segment[i:i+order])
            ngram_counts.add(ngram)
        types.append(len(ngram_counts) - prelen)
    return ngram_counts, types

In [275]:
print(get_ngrams(new2_ns_ks1_ts44[-1], 2, 5))

(Counter({((64, Fraction(1, 8)), (62, Fraction(1, 16))): 1, ((62, Fraction(1, 16)), (64, Fraction(1, 16))): 1, ((64, Fraction(1, 16)), (64, Fraction(1, 8))): 1, ((64, Fraction(1, 8)), (62, Fraction(1, 8))): 1, ((62, Fraction(1, 8)), (64, Fraction(1, 8))): 1, ((62, Fraction(1, 8)), (60, Fraction(1, 8))): 1, ((60, Fraction(1, 8)), (62, Fraction(1, 8))): 1, ((64, Fraction(1, 8)), (60, Fraction(1, 8))): 1, ((60, Fraction(1, 8)), (67, Fraction(3, 8))): 1, ((67, Fraction(3, 8)), (69, Fraction(1, 8))): 1, ((69, Fraction(1, 8)), (67, Fraction(1, 8))): 1, ((67, Fraction(1, 8)), (60, Fraction(1, 4))): 1, ((60, Fraction(1, 4)), (60, Fraction(1, 8))): 1, ((60, Fraction(1, 8)), (65, Fraction(1, 8))): 1, ((65, Fraction(1, 8)), (64, Fraction(1, 8))): 1, ((64, Fraction(1, 8)), (65, Fraction(1, 8))): 1, ((65, Fraction(1, 8)), (67, Fraction(1, 8))): 1, ((67, Fraction(1, 8)), (64, Fraction(1, 4))): 1, ((64, Fraction(1, 4)), (62, Fraction(1, 4))): 1, ((62, Fraction(1, 4)), (64, Fraction(3, 8))): 1, ((64, 

In [278]:
dview.execute(
'''
def std_compute_bleu(candidate, merged_ref_ngram_counts, ave_ref_ngram_types, minn, maxn, smooth):
    can_lent = len(candidate)
    if(maxn > can_lent):
        print("too short input!")
        return (0.0, [])
    
    
    matches_by_order = [0] * (maxn+1)

 

        
    candidate_ngram_counts, candidate_ngram_types = get_ngrams(candidate, minn, maxn)
    
    overlap = merged_ref_ngram_counts.intersection(candidate_ngram_counts)
    for ngram in overlap:
        matches_by_order[len(ngram)] += 1
        
    
    
    precisions = [0.0] * (maxn+1)
    bp = [0.0] * (maxn+1)
    
    for n in range(minn, maxn+1):
        if smooth:
            precisions[n] = (matches_by_order[n]+1.) / (candidate_ngram_types[n-minn] + 1.)
        else:
            precisions[n] = float(matches_by_order[n]) / candidate_ngram_types[n-minn]
             
        ratio = float( candidate_ngram_types[n-minn] ) / ave_ref_ngram_types[n-minn]
        
        if ratio > 1.0:
            bp[n] = 1.
        else:
            bp[n] = math.exp(1 - 1. / ratio)
        precisions[n] *= bp[n]
    
        
    
    precisions = precisions[minn:]
    bp = bp[minn:]
 
    n_sum = sum(range(minn, maxn+1))
    weight = [float(i) / n_sum for i in range(minn, maxn+1)]
    if min(precisions) > 0:
        p_log_sum = sum(w * math.log(p) for w, p in zip(weight, precisions))
        bleu = math.exp(p_log_sum)
    else:
        bleu = 0


    return (-math.log(bleu) if bleu > 0 else 100000000.0, list(zip(precisions, bp)))
''')

<AsyncResult: execute:finished>

In [17]:
dview.execute(
'''
def ave_compute_bleu(candidate, ref_ngram_counts_list, min_ref_len, minn, maxn, smooth, advanced = False):
    can_lent = len(candidate)
    if(maxn > can_lent):
        print("too short input!")
        return (0.0, [])
    
    possible_matches_by_order = [0] * (maxn+1)
    for n in range(minn, maxn+1):
        possible_matches_by_order[n] = can_lent - n + 1
    
    candidate_ngram_counts = get_ngrams(candidate, minn, maxn)
    
    precisions = [0] * (maxn+1)
    
    for ref_ngram_counts in ref_ngram_counts_list:
        overlap = candidate_ngram_counts & ref_ngram_counts
        
        matches_by_order = [0] * (maxn+1)
        for ngram in overlap:
            matches_by_order[len(ngram)] += overlap[ngram]
        
        for n in range(minn, maxn+1):
            if smooth:
                precisions[n] += (matches_by_order[n]+1.) / (possible_matches_by_order[n] + 1.)
            else:
                precisions[n] += float(matches_by_order[n]) / possible_matches_by_order[n]
    
    for n in range(minn, maxn + 1):
        precisions[n] /= len(ref_ngram_counts_list)
    
    precisions = precisions[minn:]
    
    if min(precisions) > 0:
        p_log_sum = sum((1. / maxn) * math.log(p) for p in precisions)
        geo_mean = math.exp(p_log_sum)
    else:
        geo_mean = 0

    
    ratio = float(can_lent if not advanced else len(candidate_ngram_counts)) / min_ref_len

    if ratio > 1.0:
        bp = 1.
    else:
        bp = math.exp(1 - 1. / ratio)

    bleu = geo_mean * bp

    return (-math.log(bleu) if bleu > 0 else -1, precisions)
''')

<AsyncResult: execute:finished>

In [258]:
def get_all(candidate):
    return (candidate[0], std_compute_bleu(candidate[1], candidate[5], candidate[6], candidate[2], candidate[3], candidate[4]))

In [256]:
def merge_and_sum(x, y):
    return (x[0].union(y[0]), list(map(lambda a, b: a+b, x[1],y[1])))

In [287]:
def api_get_ngrams(ref):
    return get_ngrams(ref[0], ref[1], ref[2])

In [285]:
def compute_multi_bleu(candidate_list, ref_list, minn = 2, maxn = 5, smooth = True):
    ref_ngram_list = dview.map(api_get_ngrams, [(ref, minn, maxn) for ref in ref_list])
   
    print("Done Reference!")
    merged_ref_ngram_counts, ave_ref_ngram_types = reduce(merge_and_sum, ref_ngram_list)
    
    print("Done Reduce!")
    ave_ref_ngram_types = [float(x) / len(ref_list) for x in ave_ref_ngram_types]
       
    candidate_list = [(x[0], x[1], minn, maxn, smooth, merged_ref_ngram_counts, ave_ref_ngram_types) for x in candidate_list]
    
    print("Done prework!")
    return dview.map(get_all, candidate_list)

In [311]:
def final_get_ngrams(segment, minn, maxn):
    ngram_counts = collections.Counter()
    for n in range(minn, maxn+1):
        for i in list(range(len(segment)))[::4]: # the length is 4n+1 
            if(i + 4*n+1 > len(segment)):
                break

            cur = i
            while(cur < i + 4*n+1 and segment[cur] <= 0):
                cur += 1
            modify = 0
            if(cur < i + 4*n+1):
                while(segment[cur] + modify < 60):
                    modify += 12
                while(segment[cur] + modify >= 72):
                    modify -= 12
            ngram = tuple(x if x <= 0 else x + modify for x in segment[i:i+4*n+1])
            ngram_counts[ngram] += 1
    return ngram_counts
            
dview.execute('''
def final_get_ngrams(segment, minn, maxn):
    ngram_counts = collections.Counter()
    for n in range(minn, maxn+1):
        for i in list(range(len(segment)))[::4]: # the length is 4n+1 
            if(i + 4*n+1 > len(segment)):
                break

            cur = i
            while(cur < i + 4*n+1 and segment[cur] <= 0):
                cur += 1
            modify = 0
            if(cur < i + 4*n+1):
                while(segment[cur] + modify < 60):
                    modify += 12
                while(segment[cur] + modify >= 72):
                    modify -= 12
            ngram = tuple(x if x <= 0 else x + modify for x in segment[i:i+4*n+1])
            ngram_counts[ngram] += 1
    return ngram_counts
''')
    

<AsyncResult: execute:finished>

In [130]:
def new_compute_multi_bleu(candidate_list, ref_list, minn = 1, maxn = 2, smooth = False):
    
    ngram_counts = collections.Counter()
    
    
    ref_ngram_counts_list = []

    ref_list = [x[::4] for x in ref_list]
    
    
    for ref in ref_list:
        ref_ngram_counts_list.append(get_ngrams(ref, minn, maxn))
       
    
    merged_ref_ngram_counts = collections.Counter()
    for i in range(0, len(ref_ngram_counts_list)):
        merged_ref_ngram_counts |= ref_ngram_counts_list[i]
    
    min_ref_len = min([int(len(x) / 4) for x in ref_list]) 
    candidate_list = [(x[0], x[1][::4], minn, maxn, smooth, merged_ref_ngram_counts,  min_ref_len) for x in candidate_list]
    
   
    return dview.map(get_all, candidate_list)

In [156]:
zjl = []
non_zjl = []
for i in range(0, len(ns_ks1_ts44)):
    if "周杰伦" in ns_ks1_ts44[i].filename:
        zjl.append(i)
    else:
        non_zjl.append(i)
        
print(len(zjl))
print(len(non_zjl))

87
2048


In [309]:
random.shuffle(non_zjl)
sample_list2 = [final_ns_ks1_ts44[i] for i in non_zjl[:1000]]

In [300]:
import random
def get_random_melody():
    random_ns = []
    now_index = 5
    random_list = [52,53,55,57,59,60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77, 79, 81]


    for i in range(0, 2000):
        choice = random.randint(0, 6)
        if(choice < 2):
            step = 0
        elif(choice < 5):
            step = 1
        else:
            step = 2
        if(now_index < 2):
            random_ns.append((random_list[now_index], Fraction(1, 8)))
            now_index += step
        elif(now_index > 15):
            random_ns.append((random_list[now_index], Fraction(1, 8)))
            now_index -= step
        else: # index is in [2, 15]
            if(random.randint(2, 17) > now_index):
                random_ns.append((random_list[now_index], Fraction(1, 8)))
                now_index += step
            else:
                random_ns.append((random_list[now_index], Fraction(1, 8)))
                now_index -= step
        
    random_ns.append((random_list[now_index], Fraction(1, 8)))
    change_list = ['C', '#C', 'D', '#D', 'E', 'F', '#F', 'G', '#G', 'A', 'bB', 'B']
    for pitchh, time in random_ns:
        while(pitchh < 0):
            pitchh += 12
        pitchh %= 12
        print(change_list[pitchh])
    return random_ns

## test_random

In [301]:
my_random1 = get_random_melody()
print(my_random1)

C
E
C
D
D
F
G
B
A
B
G
A
A
F
F
G
A
B
B
G
A
A
C
A
A
G
G
A
G
F
F
F
G
G
A
B
A
G
G
F
D
D
E
E
E
E
E
E
C
D
D
E
E
E
D
E
G
E
F
F
A
F
D
C
E
E
D
B
C
D
D
B
C
E
C
C
B
A
A
B
B
D
E
C
D
F
E
F
F
F
G
A
C
B
C
B
B
B
A
A
B
A
G
F
F
F
E
F
G
F
F
G
E
C
D
C
B
D
D
F
E
F
G
G
A
A
F
E
G
A
C
A
A
B
A
F
D
D
E
D
D
F
E
E
G
G
F
G
G
B
B
B
C
D
B
G
F
G
B
B
D
C
D
C
D
E
D
E
C
C
C
A
G
B
G
F
G
B
A
A
G
F
G
F
A
A
G
F
E
D
F
A
F
F
D
E
C
D
E
G
B
B
C
C
E
E
F
F
D
E
C
C
B
A
G
F
E
E
D
E
D
F
F
G
G
F
G
F
F
G
G
G
A
A
G
B
B
G
B
A
G
G
F
F
F
D
D
C
D
E
G
G
B
C
C
D
C
B
B
A
C
B
A
G
B
B
B
G
G
F
F
G
A
F
E
F
G
B
D
C
E
C
B
B
B
A
A
C
D
D
B
C
A
A
G
A
C
A
C
C
C
C
C
C
A
B
G
A
A
F
E
F
F
F
F
G
F
F
A
G
F
G
A
B
G
A
B
B
C
C
B
C
C
A
F
G
E
F
F
D
D
C
B
B
C
E
E
E
F
A
B
A
B
A
G
F
G
A
F
D
D
C
D
C
D
E
E
G
E
C
D
F
E
E
D
B
C
E
D
D
D
C
B
C
C
B
D
C
D
C
B
C
A
A
B
B
B
A
C
D
E
C
C
D
D
D
F
F
A
B
G
F
A
G
G
G
G
B
A
A
G
A
C
C
E
D
F
F
E
D
C
B
C
C
B
C
A
A
A
F
E
G
A
C
B
B
A
A
A
G
G
B
C
A
C
A
G
B
D
D
B
D
D
B
B
B
D
D
E
C
E
E
E
C
C
C
E
E
E
C
D
D
E
D
B
A
B
A
C
C
E
E
D
D
D
B
C
E
F
E


In [134]:
my_new_random1 = [0] * 1025
for i in range(0, 256):
    my_new_random1[i*4] = my_random1[i]
    my_new_random1[i*4+1] = my_random1[i]
    my_new_random1[i*4+2] = my_random1[i]
    my_new_random1[i*4+3] = my_random1[i]
    
print(my_new_random1)   

[60, 60, 60, 60, 60, 60, 60, 60, 64, 64, 64, 64, 62, 62, 62, 62, 62, 62, 62, 62, 64, 64, 64, 64, 64, 64, 64, 64, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 71, 71, 71, 71, 67, 67, 67, 67, 67, 67, 67, 67, 69, 69, 69, 69, 72, 72, 72, 72, 74, 74, 74, 74, 71, 71, 71, 71, 71, 71, 71, 71, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 74, 74, 74, 74, 72, 72, 72, 72, 76, 76, 76, 76, 74, 74, 74, 74, 72, 72, 72, 72, 74, 74, 74, 74, 72, 72, 72, 72, 72, 72, 72, 72, 74, 74, 74, 74, 71, 71, 71, 71, 69, 69, 69, 69, 65, 65, 65, 65, 67, 67, 67, 67, 67, 67, 67, 67, 69, 69, 69, 69, 69, 69, 69, 69, 65, 65, 65, 65, 67, 67, 67, 67, 67, 67, 67, 67, 69, 69, 69, 69, 72, 72, 72, 72, 76, 76, 76, 76, 76, 76, 76, 76, 72, 72, 72, 72, 71, 71, 71, 71, 71, 71, 71, 71, 72, 72, 72, 72, 72, 72, 72, 72, 69, 69, 69, 69, 67, 67, 67, 67, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 67, 67, 67, 67, 67, 67, 67, 67, 65, 65, 65, 65, 64, 64, 64, 64, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 59, 59, 59, 59, 60, 60,

## get_ranks

In [58]:
def get_brief_ranks(myseq):

    myseq.sort(key = std_bleu)

    for i in range(0, len(myseq)):
#         print(i+1)
#         for m in myseq[i]:
#             print(m)
         print(str(i+1) + ': ' + myseq[i][0])
#         print('--------')

    print('\n\n\n')



In [57]:
def std_bleu(element):
    return element[1]

In [59]:
def get_full_ranks(myseq):

    myseq.sort(key = std_bleu)

    for i in range(0, len(myseq)):
        print(i+1)
        for m in myseq[i]:
            print(m)
        #print(str(i+1) + ': ' + myseq[i][0])
        print('--------')

    print('\n\n\n')


In [261]:
sample_list_num = random.sample(non_zjl, 1000)

In [262]:
sample_list2 = [new2_ns_ks1_ts44[i] for i in sample_list_num]

In [70]:
for i in zjl:
    if('晴天' in ns_ks1_ts44[i].filename):
        print(len(final_ns_ks1_ts44[i]))
        print(final_ns_ks1_ts44[i])

1056
[0, 0, 0, 0, 67, 67, 67, 67, 67, 67, 67, 67, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 62, 62, 62, 62, 64, 64, 64, 64, 64, 64, 64, 64, 67, 67, 67, 67, 67, 67, 67, 67, 60, 60, 60, 60, 60, 60, 60, 60, 62, 62, 64, 64, 62, 62, 60, 60, 55, 55, 55, 55, 0, 0, 0, 0, 67, 67, 67, 67, 67, 67, 67, 67, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 62, 62, 62, 62, 64, 64, 64, 64, 0, 0, 0, 0, 64, 64, 64, 64, 64, 64, 64, 64, 55, 55, 65, 65, 65, 65, 64, 64, 62, 62, 65, 65, 64, 64, 62, 62, 60, 60, 60, 60, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 65, 65, 65, 65, 64, 64, 64, 64, 62, 62, 62, 62, 60, 60, 62, 62, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 62, 62, 64, 64, 62, 62, 60, 60, 59, 59, 59, 59, 59, 59, 59, 59, 55, 55, 55, 55, 60, 60, 60, 60, 60, 60, 60, 60, 64, 64, 64, 64, 65, 65, 65, 65, 64, 64, 64, 64, 62, 62, 62, 62, 60, 60, 62, 62, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 62, 62, 64, 64, 62, 62, 62, 62, 60, 60, 60, 60

In [232]:
candidate_machine_sample = random.sample(range(8001), 1000)
print(candidate_machine_sample)

[6085, 5487, 800, 2393, 4107, 5732, 7430, 6197, 5546, 4145, 5470, 5957, 183, 5929, 1345, 3994, 4314, 2429, 6598, 4994, 4274, 56, 5385, 2116, 1742, 6545, 4105, 5935, 7221, 4964, 3504, 5234, 6106, 4472, 5598, 3100, 4267, 5432, 3166, 1314, 5102, 5471, 5096, 5022, 995, 245, 3508, 865, 7760, 3845, 1584, 1078, 2035, 4742, 6434, 4564, 1342, 2501, 7447, 6087, 3743, 6249, 1680, 6937, 2390, 3444, 5812, 6705, 6851, 7886, 5841, 4549, 2590, 5120, 1903, 5881, 5901, 271, 6731, 5024, 310, 1005, 4021, 7655, 5352, 6343, 7412, 2316, 2309, 3272, 3077, 5973, 2747, 997, 4392, 536, 6890, 5437, 6796, 7831, 7883, 4428, 2682, 4893, 6239, 4263, 3946, 6030, 3865, 1934, 7813, 2156, 1279, 6607, 2960, 2493, 3278, 7875, 4916, 7751, 2268, 6986, 198, 2655, 2100, 6634, 4923, 3901, 6706, 5084, 3436, 1383, 6181, 3611, 238, 895, 2203, 2228, 5986, 2001, 6072, 5918, 3899, 3342, 6690, 1714, 2810, 2486, 48, 3047, 7002, 3232, 7223, 2514, 2676, 554, 670, 1898, 2904, 7607, 2210, 2806, 5016, 4154, 5780, 1433, 1597, 7368, 4832, 495

In [310]:
res2 = new_compute_multi_bleu([("ruanruan1", my_new_random1)] + [(ns_ks1_ts44[i].filename, final_ns_ks1_ts44[i]) for i in (non_zjl[1000:] + zjl)] + [(machine_sequence_list[i].filename, final_machine_ns[i]) for i in candidate_machine_sample], sample_list2)

TypeError: 'int' object is not subscriptable

In [305]:
old_res = compute_multi_bleu([("ruanruan", my_random1[:230])]+[(ns_ks1_ts44[i].filename, new2_ns_ks1_ts44[i]) for i in (non_zjl[1000:] + zjl)] + [(machine_sequence_list[i].filename, machine_ns[i]) for i in candidate_machine_sample], sample_list2)

Done Reference!
Done Reduce!
Done prework!


In [102]:
for i in zjl:
    for note in ns_ks1_ts44[i].notes:
        if(note.denominator % 3 == 0):
            print(ns_ks1_ts44[i].filename)
 

225 - 周杰伦 - 给我一首歌的时间.xml
225 - 周杰伦 - 给我一首歌的时间.xml
225 - 周杰伦 - 给我一首歌的时间.xml
625 - 周杰伦 - 暗号.xml
625 - 周杰伦 - 暗号.xml
625 - 周杰伦 - 暗号.xml
625 - 周杰伦 - 暗号.xml
625 - 周杰伦 - 暗号.xml
625 - 周杰伦 - 暗号.xml
44 - 周杰伦 - 说好的幸福呢.xml
44 - 周杰伦 - 说好的幸福呢.xml
44 - 周杰伦 - 说好的幸福呢.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦 - 米兰的小铁匠.xml
717 - 周杰伦

In [307]:
get_full_ranks(res2)

1
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_basic/5/5-1530161419.2428741.xml
(-1, [0.109375, 0.0])
--------
2
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_lookback/0/2-1530160693.5645912.xml
(-1, [0.109375, 0.0])
--------
3
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_onehot/44/0-1530266701.3650827.xml
(-1, [0.125, 0.0])
--------
4
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_onehot/65/5-1530269780.766994.xml
(-1, [0.125, 0.0])
--------
5
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_basic/22/5-1530163966.0547142.xml
(-1, [0.125, 0.0])
--------
6
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_basic/55/3-1530168939.758996.xml
(-1, [0.1346153846153846, 0.0])
--------
7
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_onehot/43/5-1530266554.2981462.xml
(-1, [0.140625, 0.0])
--------
8
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_lookback/93/7-1530268202.8677423.xml
(-1, [0.140625, 0.0])
---

(0.8683436197428097, [0.8125, 0.30158730158730157])
--------
130
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/68/5-1530098725.0695503.xml
(0.8690188823074586, [0.875, 0.2903225806451613])
--------
131
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/24/5-1530096551.872304.xml
(0.8690188823074586, [0.875, 0.2903225806451613])
--------
132
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_lookback/46/9-1530261454.66998.xml
(0.8748163150285102, [0.796875, 0.30158730158730157])
--------
133
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_lookback/63/6-1530170533.2257767.xml
(0.8789210328245968, [0.9375, 0.3333333333333333])
--------
134
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_lookback/98/1-1530268917.1226084.xml
(0.8856919450393124, [0.859375, 0.2857142857142857])
--------
135
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_lookback/41/6-1530256585.3923578.xml
(0.8950245223006552, [0.75, 0.30158730158730157])

(1.1447123844176437, [0.64, 0.22448979591836735])
--------
273
1837 - 河原ゆう - トキメキスマイルステージ.xml
(1.1462517597664585, [0.7352941176470589, 0.208955223880597])
--------
274
2723 - 胡彦斌Tiger Hu - Waiting For You.xml
(1.1478439608537943, [0.7567567567567568, 0.2054794520547945])
--------
275
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/74/5-1530099023.628751.xml
(1.1487192467018006, [0.625, 0.22580645161290322])
--------
276
1839 - mammy Sino - 母の想ひ.xml
(1.14983383969685, [0.7058823529411765, 0.21212121212121213])
--------
277
1152 - 梁静茹 - 纯真.xml
(1.1511552700693042, [0.640625, 0.2222222222222222])
--------
278
1556 - 南里侑香 - 暁の车.xml
(1.1513666127233133, [0.7083333333333334, 0.2112676056338028])
--------
279
1421 - 上原あずみ - 無色.xml
(1.1513666127233133, [0.7083333333333334, 0.2112676056338028])
--------
280
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_onehot/50/3-1530267578.771817.xml
(1.1579013574829677, [0.546875, 0.23809523809523808])
--------
281
2252 - 林宥嘉 - 宠

(1.2960539582841446, [0.671875, 0.1746031746031746])
--------
416
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/37/6-1530261891.5349436.xml
(1.2960539582841446, [0.671875, 0.1746031746031746])
--------
417
510 - 范玮琪 - 对的人.xml
(1.2965367479747438, [0.5909090909090909, 0.18604651162790697])
--------
418
1960 - 梶浦由記 - Dream☆Wing.xml
(1.2968052318885974, [0.6481481481481481, 0.17757009345794392])
--------
419
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/6-1527854656.7626424.xml
(1.2971114237898629, [0.6176470588235294, 0.18181818181818182])
--------
420
2567 - 周传雄 - 不一样.xml
(1.2972734327035422, [0.6944444444444444, 0.17142857142857143])
--------
421
2370 - 胡彦斌Tiger Hu - 成长.xml
(1.2972734327035422, [0.6944444444444444, 0.17142857142857143])
--------
422
1033 - 李健 - 我愿人长久.xml
(1.2993100274022893, [0.5714285714285714, 0.18840579710144928])
--------
423
1925 - 朝川弘子 - Moonlight Destiny.xml
(1.30286907219075, [0.7631578947368421, 0.1621621621621621

/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/45/8-1530262321.0033765.xml
(1.4073539949937346, [0.71875, 0.14285714285714285])
--------
561
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_lookback/87/5-1530259027.6063175.xml
(1.4073539949937346, [0.71875, 0.14285714285714285])
--------
562
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/8/1-1530095764.7405336.xml
(1.4081542430018845, [0.5625, 0.16129032258064516])
--------
563
615 - 林俊杰 - 完美新世界.xml
(1.4087637406036717, [0.5405405405405406, 0.1643835616438356])
--------
564
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_lookback/91/5-1530259238.9763193.xml
(1.4096881458368071, [0.578125, 0.15873015873015872])
--------
565
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/68/5-1530263557.0314188.xml
(1.4096881458368071, [0.578125, 0.15873015873015872])
--------
566
425 - 林俊杰,G.E.M.xml
(1.411689532118303, [0.6826923076923077, 0.14563106796116504])
--------

...

/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/86/9-1530099614.0478036.xml
(1.7879656707313394, [0.5, 0.0967741935483871])
--------
1171
788 - 李健 - 完美坚持.xml
(1.78830676253436, [0.6458333333333334, 0.0851063829787234])
--------
1172
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/13/2-1530260611.1307871.xml
(1.7883753454067155, [0.515625, 0.09523809523809523])
--------
1173
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_onehot/42/2-1530266409.102527.xml
(1.7883753454067155, [0.515625, 0.09523809523809523])
--------
1174
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_onehot/53/1-1530268022.43009.xml
(1.7883753454067155, [0.515625, 0.09523809523809523])
--------
1175
1665 - 榊原ゆい - リフレイン (エンディングテーマ).xml
(1.7884855564618882, [0.39285714285714285, 0.10909090909090909])
--------
1176
519 - 张杰 - 北斗星的爱.xml
(1.7900212151910333, [0.45, 0.1016949152542373])
--------
1177
1239 - G.E.M.xml
(1.791205035612449, [0.43333333333333335, 0.10344827586

1334
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_lookback/37/6-1530256371.8752785.xml
(1.9201802694916033, [0.5, 0.07936507936507936])
--------
1335
2053 - mizuki,SawanoHiroyuki[nZk] - A／Z.xml
(1.9206503388143206, [0.573170731707317, 0.07407407407407407])
--------
1336
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_lookback/59/4-1530169909.9006271.xml
(1.920903486179013, [0.46774193548387094, 0.08196721311475409])
--------
1337
1764 - MAGIC OF LiFE - 桜川.xml
(1.9239418832608168, [0.47368421052631576, 0.08108108108108109])
--------
1338
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/2-1527854357.4032767.xml
(1.9239418832608168, [0.47368421052631576, 0.08108108108108109])
--------
1339
1537 - いとうかなこ - 追想のディスペア.xml
(1.926828760354741, [0.5217391304347826, 0.07692307692307693])
--------
1340
2066 - Akie秋绘 - 天ノ弱.xml
(1.9269145052641097, [0.25, 0.1111111111111111])
--------
1341
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_05

1507
1953 - Sotte Bosse - 夜空ノムコウ.xml
(2.0607738737072694, [0.5454545454545454, 0.06153846153846154])
--------
1508
412 - 萧煌奇 - 孤独的和弦.xml
(2.0613530803401163, [0.3484848484848485, 0.07692307692307693])
--------
1509
248 - 五月天 - 倔强.xml
(2.062880769087597, [0.5588235294117647, 0.06060606060606061])
--------
1510
723 - 林俊杰 - 莎士比亚的天份.xml
(2.062880769087597, [0.5588235294117647, 0.06060606060606061])
--------
1511
1422 - fripSide - only my railgun.xml
(2.063525195126162, [0.21875, 0.0967741935483871])
--------
1512
716 - 范玮琪 - 不眠.xml
(2.0648017970348906, [0.2777777777777778, 0.08571428571428572])
--------
1513
2512 - 张韶涵 - 不害怕.xml
(2.0650001507053277, [0.38095238095238093, 0.07317073170731707])
--------
1514
1671 - 柴田淳 - 梦.xml
(2.0654059355479477, [0.5, 0.06382978723404255])
--------
1515
1818 - ChouCho - 優しさの理由.xml
(2.0654059355479477, [0.5, 0.06382978723404255])
--------
1516
1336 - fripSide - sister's noise.xml
(2.0669276030962056, [0.4069767441860465, 0.07058823529411765])
--------
1517


1700
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/90/2-1530264744.9837897.xml
(2.239996787507372, [0.4666666666666667, 0.05084745762711865])
--------
1701
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/15/7-1530260718.5406067.xml
(2.2405224780634523, [0.53125, 0.047619047619047616])
--------
1702
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_lookback/70/3-1530171629.3388507.xml
(2.242432882932565, [0.34615384615384615, 0.058823529411764705])
--------
1703
1260 - 李荣浩 - 流行歌曲.xml
(2.2434289306124184, [0.5606060606060606, 0.046153846153846156])
--------
1704
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_lookback/26/7-1530164731.9848535.xml
(2.2440370052417142, [0.1774193548387097, 0.08196721311475409])
--------
1705
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_basic/11/7-1530162317.1865926.xml
(2.2440370052417142, [0.1774193548387097, 0.08196721311475409])
--------
1706
935 - 汪苏泷 - 巴赫旧约.xml
(2.2446656

1870
2345 - 张韶涵 - 静不下来.xml
(2.561266078732675, [0.7, 0.02564102564102564])
--------
1871
739 - 徐良 - 他的猫.xml
(2.562054101164302, [0.5, 0.030303030303030304])
--------
1872
1112 - 范玮琪 - 长头发.xml
(2.562054101164302, [0.5, 0.030303030303030304])
--------
1873
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_lookback/31/9-1530256055.0401924.xml
(2.563854115012124, [0.453125, 0.031746031746031744])
--------
1874
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/88/7-1530264636.513493.xml
(2.563854115012124, [0.453125, 0.031746031746031744])
--------
1875
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_onehot/63/2-1530269490.82374.xml
(2.563854115012124, [0.453125, 0.031746031746031744])
--------
1876
401 - 许嵩 - 两种悲剧.xml
(2.56488432738492, [0.4375, 0.03225806451612903])
--------
1877
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/31/2-1530096899.2304573.xml
(2.56488432738492, [0.4375, 0.03225806451612903])
--------
1878
/home/srthu2/dongyu

In [306]:
get_full_ranks(old_res)

1
1448 - Duca - Melody.xml
(0.12431584574080883, [(1.0, 1.0), (0.9726027397260274, 1.0), (0.8734177215189873, 1.0), (0.8, 1.0)])
--------
2
2084 - Suara,藤田淳平 - 星座.xml
(0.16674684407565676, [(0.9905660377358491, 1.0), (0.9609375, 1.0), (0.8368794326241135, 1.0), (0.7432432432432432, 1.0)])
--------
3
2367 - 莫文蔚 - 午夜前的十分钟.xml
(0.17761774879256673, [(0.9597288707103854, 0.9820481467734176), (0.930654574189075, 0.9878000304989305), (0.8333333333333334, 1.0), (0.7468354430379747, 1.0)])
--------
4
1724 - 東山奈央 - 想い出がいっぱい.xml
(0.1996986374800112, [(1.0, 1.0), (0.9523809523809523, 1.0), (0.841726618705036, 1.0), (0.6756756756756757, 1.0)])
--------
5
2569 - 胡彦斌 - 情不自禁.xml
(0.2030359534441735, [(0.9469810557574027, 0.9469810557574027), (0.9797690553842963, 0.9960985396407012), (0.8119311980900628, 0.9801691940907065), (0.6922726357372865, 0.9761044163895739)])
--------
6
622 - 周杰伦 - 开不了口.xml
(0.2054514045662259, [(0.898345985825023, 0.898345985825023), (0.9796363938832369, 0.9878000304989305)

(0.4515616907379396, [(0.9893617021276596, 1.0), (0.9020979020979021, 1.0), (0.6923076923076923, 1.0), (0.4049079754601227, 1.0)])
--------
137
850 - 周杰伦 - Mine Mine.xml
(0.4518945164798453, [(0.9844961240310077, 1.0), (0.8956521739130435, 1.0), (0.6743421052631579, 1.0), (0.4157303370786517, 1.0)])
--------
138
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/66/6-1530263450.4356937.xml
(0.4523682141684002, [(0.8539832210929231, 0.898345985825023), (0.8936170212765957, 1.0), (0.686046511627907, 1.0), (0.4340659340659341, 1.0)])
--------
139
1153 - 周杰伦 - 心雨.xml
(0.4528474601877067, [(1.0, 1.0), (0.938945344743284, 0.9960985396407012), (0.7024189818259262, 0.9950935575867287), (0.3876726760128804, 0.9761044163895739)])
--------
140
1346 - 石川智晶 - アンインストール(テレビアニメ＂ぼくらの＂Op).xml
(0.4528504873742321, [(0.989010989010989, 1.0), (0.8832116788321168, 1.0), (0.6687898089171974, 1.0), (0.42011834319526625, 1.0)])
--------
141
1733 - 金泰贤 - 내맘이 그래요.xml
(0.4542842638010548

(0.5408985276497933, [(1.0, 1.0), (0.8970588235294118, 1.0), (0.6265822784810127, 1.0), (0.3411764705882353, 1.0)])
--------
275
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/85/9-1530264473.1916573.xml
(0.5410812687309539, [(0.9694656488549618, 1.0), (0.8404255319148937, 1.0), (0.6431924882629108, 1.0), (0.3515981735159817, 1.0)])
--------
276
2720 - 杨丞琳 - 绝对达令.xml
(0.5417099447433961, [(0.9823132694910665, 0.9934759202807377), (0.9197080291970803, 1.0), (0.7012987012987013, 1.0), (0.30864197530864196, 1.0)])
--------
277
1423 - AKB48 - 誰かが投げたボール.xml
(0.5422022350619271, [(1.0, 1.0), (0.9155844155844156, 1.0), (0.6607142857142857, 1.0), (0.3218390804597701, 1.0)])
--------
278
2389 - 刘思涵 - 不懂爱.xml
(0.5422348515826508, [(1.0, 1.0), (0.916083916083916, 1.0), (0.6273291925465838, 1.0), (0.33532934131736525, 1.0)])
--------
279
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/80/5-1530264203.2429583.xml
(0.5424927803659514, [(0.9603960396039604,

419
375 - 张靓颖 - 为难.xml
(0.6188856681906411, [(0.8212128279540252, 0.8212128279540252), (0.7508452293279414, 0.8067592357672562), (0.5552174765695712, 0.782999005418626), (0.36369454448214583, 0.7812697622209059)])
--------
420
103 - 薛之谦 - 苏黎世的从前.xml
(0.6189431583993357, [(0.8102633235813048, 0.8212128279540252), (0.7219271231817364, 0.8262054854190983), (0.5867970842909492, 0.8693290137643691), (0.3581072217282661, 0.9021547316615934)])
--------
421
1308 - スフィア - 優しさに包まれるように.xml
(0.6191200349460623, [(0.9459459459459459, 1.0), (0.8098591549295775, 1.0), (0.5714285714285714, 1.0), (0.32075471698113206, 1.0)])
--------
422
227 - 林俊杰 - 豆浆油条.xml
(0.6193517927258853, [(0.9934759202807377, 0.9934759202807377), (0.8968253968253969, 1.0), (0.6137931034482759, 1.0), (0.2792207792207792, 1.0)])
--------
423
204 - 张惠妹 - 我恨我爱你.xml
(0.6209079095697849, [(1.0, 1.0), (0.8646103814565267, 0.9624908019987749), (0.5618921707406033, 0.9340545175947691), (0.30419080834578444, 0.9471395623493742)])
-------

--------
572
839 - 张靓颖 - 梦想.xml
(0.7086585281295334, [(0.922936243695492, 0.922936243695492), (0.8311837957920041, 0.9452678461948282), (0.5910813744154398, 0.9340545175947691), (0.2415733889589351, 0.9248809748713516)])
--------
573
555 - 孙燕姿 - 一起走到.xml
(0.708674751775628, [(0.9807692307692307, 1.0), (0.8633093525179856, 1.0), (0.5870967741935483, 1.0), (0.23170731707317074, 1.0)])
--------
574
1896 - 赵贤雅 - 사는게 아니야.xml
(0.708717457760325, [(0.9801980198019802, 1.0), (0.8712121212121212, 1.0), (0.5144927536231884, 1.0), (0.256140875222796, 0.9761044163895739)])
--------
575
1696 - JUJU - どんなに遠くても... ~Game Ending ver.xml
(0.7089922716321783, [(0.968, 1.0), (0.7976190476190477, 1.0), (0.5659340659340659, 1.0), (0.2513089005235602, 1.0)])
--------
576
1339 - Revo & 梶浦由記 - 砂塵の彼方へ.xml
(0.7090128336101729, [(0.9931506849315068, 1.0), (0.8960396039603961, 1.0), (0.5745614035087719, 1.0), (0.22916666666666666, 1.0)])
--------
577
1452 - AKB48 - ＃好きなんだ.xml
(0.7097848055773743, [(0.9370

...

1415
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/47/2-1530262430.3011374.xml
(1.3059951496786653, [(0.8950617283950617, 1.0), (0.594059405940594, 1.0), (0.23943661971830985, 1.0), (0.11574074074074074, 1.0)])
--------
1416
2261 - 杨丞琳 - 太烦恼.xml
(1.3060779707277297, [(0.5589393630320337, 0.5589393630320337), (0.4569180225705573, 0.5497294959052017), (0.32419276507031436, 0.6020722779877267), (0.1283174197130604, 0.648002969550955)])
--------
1417
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_lookback/82/6-1530266616.568054.xml
(1.3070526976086656, [(0.8842105263157894, 1.0), (0.5519363782526863, 0.9009549703830615), (0.3014668552202247, 0.8182671784548956), (0.10079627714768401, 0.7727714581322441)])
--------
1418
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/56/6-1530262914.3532145.xml
(1.3076540699373411, [(0.9473684210526315, 1.0), (0.7102363855653467, 0.9878000304989305), (0.31255205750012494, 0.8940442575003573), (0.081747286745

--------
1587
/home/srthu2/dongyuanliang/machine_work/bleu/data_chord_0524_onehot/13/0-1530260611.1307871.xml
(1.493425064904314, [(0.8899082568807339, 1.0), (0.6766917293233082, 1.0), (0.28527312365326535, 0.9801691940907065), (0.05517942325577761, 0.9173579116273028)])
--------
1588
2712 - 张韶涵 - 爱上爱的味道.xml
(1.4934841021385128, [(0.6307119467089208, 0.6513910269288854), (0.5062569403100299, 0.6179312653784188), (0.2809612356685407, 0.6221284504089115), (0.0762806749988892, 0.6293155687408358)])
--------
1589
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/28/3-1530096750.4949248.xml
(1.4943761517988055, [(0.7704474230335235, 0.8474921653368759), (0.4839001750954999, 0.6836050092618967), (0.21572190624216622, 0.5817954441076604), (0.08915297576685341, 0.5230307911655401)])
--------
1590
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/2-1527856561.5097206.xml
(1.4961069593502867, [(0.9696969696969697, 1.0), (0.6692791722118601, 0.9794329349441854)

693 - 许嵩 - 我想牵着你的手.xml
(1.744129823987881, [(0.37970693193817207, 0.37970693193817207), (0.2615810839135824, 0.3106275371473791), (0.15452894280549248, 0.2726981343626338), (0.11105656557189128, 0.2690986011934289)])
--------
1734
2361 - Tank - 挺你到底.xml
(1.7442394060976345, [(0.2415656034149911, 0.24809440350728815), (0.23808914100089199, 0.2628900931884849), (0.18634372514779976, 0.2726981343626338), (0.12118600055466923, 0.25908731153067216)])
--------
1735
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/74/5-1530099023.628751.xml
(1.7444979739384308, [(0.5491334092946296, 0.5589393630320337), (0.3670925247905179, 0.4314947221221877), (0.179427688562118, 0.358855377124236), (0.06931059005575381, 0.3095873022490337)])
--------
1736
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_basic/44/2-1530167266.085244.xml
(1.7449920969887764, [(0.8404255319148937, 1.0), (0.47619047619047616, 1.0), (0.22675555982695447, 0.9801691940907065), (0.04141258096438887, 0.92

927 - 李健 - 贝加尔湖畔.xml
(2.158675417638286, [(0.5079676228527362, 0.5271362123943489), (0.26762005008459305, 0.38318325353021276), (0.11427295874864996, 0.34801310164361576), (0.038886362377386095, 0.3402556708021284)])
--------
1881
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_lookback/22/5-1530258021.2420435.xml
(2.1610449732574306, [(0.8823529411764706, 1.0), (0.36728735060406953, 0.9794329349441854), (0.07987312389446417, 0.8858655559204208), (0.0341142011660044, 0.8392093486837082)])
--------
1882
367 - 周杰伦 - 夜曲.xml
(2.1642923590208545, [(0.2805005797491908, 0.2805005797491908), (0.1814681071142786, 0.19384093259934304), (0.10300224772701368, 0.14238546009322478), (0.06659418482350757, 0.12038179564249445)])
--------
1883
/home/srthu2/dongyuanliang/machine_work/bleu/Wikifonia_Chord_lookback/22/7-1530164102.9428399.xml
(2.1699655375015707, [(0.8828828828828829, 1.0), (0.4460431654676259, 1.0), (0.125, 1.0), (0.020689655172413793, 1.0)])
--------
1884
/home/srthu2/dongyuanlia

2041
2255 - 李健 - 我始终在这里(But I Still Love You).xml
(3.0563111282782955, [(0.3059028241869992, 0.31336386867936505), (0.11960397968895635, 0.19384093259934304), (0.04775752002420293, 0.17056257151501045), (0.012570033974744876, 0.1634104416716834)])
--------
2042
2344 - 刘思涵 - 梦.xml
(3.0566457985425624, [(0.3588300125192208, 0.5430940730020639), (0.18409988741888134, 0.5381381324551916), (0.032258987954237824, 0.5613063904037382), (0.012449028518432027, 0.5913288546255213)])
--------
2043
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_basic/95/8-1530100060.9147918.xml
(3.0597572846158947, [(0.3794432131425506, 0.39630735594888616), (0.15962545597285882, 0.23943818395928826), (0.051187249029845096, 0.19012406782513894), (0.009086792039230587, 0.15447546466692)])
--------
2044
/home/srthu2/dongyuanliang/machine_work/bleu/NewData_Chord_lookback_v3/0-1527853024.2469525.xml
(3.0738484628684697, [(0.7751937984496124, 1.0), (0.2517985611510791, 1.0), (0.034482758620689655, 1.0), (0.0

In [160]:
from music21 import *
c = chord.Chord("C4 E4 G4")
c.isConsonant()

True

In [161]:
import notebook
notebook.nbextensions.check_nbextension('usability/sphinx-markdown', user=True)

False

In [170]:

from music21 import *
b = converter.parse(mxl_files[-1])
b.show('midi') # I've altered this so it's much shorter than it should be...